In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('sentiwordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import sentiwordnet as swn

from nltk.sentiment.vader import SentimentIntensityAnalyzer

import stanza
stanza.download('en')  # Download the English model

import statistics
import numpy as np
import pandas as pd

import re

import requests
from bs4 import BeautifulSoup

import newspaper

[nltk_data] Downloading package punkt to /home/pierluigi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/pierluigi/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


2023-04-20 15:26:42 INFO: Downloading default packages for language: en (English) ...
2023-04-20 15:26:44 INFO: File exists: /home/pierluigi/stanza_resources/en/default.zip
2023-04-20 15:26:49 INFO: Finished downloading models and saved to /home/pierluigi/stanza_resources.


In [6]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment', tokenize_no_ssplit=False, max_split_size_mb=128)

2023-04-20 15:52:35 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-04-20 15:52:36 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2023-04-20 15:52:36 INFO: Using device: cuda
2023-04-20 15:52:36 INFO: Loading: tokenize
2023-04-20 15:52:38 INFO: Loading: sentiment
2023-04-20 15:52:39 INFO: Done loading processors!


In [7]:
# Load the MPQA lexicon
lexicon = pd.read_csv("subjclueslen1-HLTEMNLP05.tff", sep=" ", header=None, 
                      names=["type", "len", "word", "pos", "stemmed", "polarity", "strength"])

lexicon["type"] = lexicon["type"].str[5:]
lexicon["word"] = lexicon["word"].str[len("word1="):]
lexicon["polarity"] = lexicon["polarity"].str[len("priorpolarity="):]
cols_to_remove = ["len", "pos", "stemmed", "strength"]
lexicon = lexicon.drop(columns=cols_to_remove)
lexicon["type"] = lexicon["type"].replace("weaksubj", 1)
lexicon["type"] = lexicon["type"].replace("strongsubj", 2)
lexicon["polarity"] = lexicon["polarity"].replace("negative", -1)
lexicon["polarity"] = lexicon["polarity"].replace("positive", 1)
lexicon["polarity"] = lexicon["polarity"].replace("both", 0)
lexicon["polarity"] = lexicon["polarity"].replace("neutral", 0)

In [ ]:
url = "https://www.foxnews.com/politics/republicans-respond-after-irs-whistleblower-says-hunter-biden-investigation-being-mishandled"

num_stop_words_per_sentence = []
stop_words_per_sentence = []
filtered_sentences = []
num_words_per_sentence = []
avg_stop_words_per_sentence = []
total_words = 0
results = []
scores_list = []
scores = []

In [9]:
def preprocess_article(url):
    # Create a newspaper Article object
    article = newspaper.Article(url)

    # Download and parse the article
    article.download()
    article.parse()

    # Extract the title, subtitle, description, and main text
    title = article.title.strip()
    subtitle = article.meta_data.get("description", "").strip()
    description = article.meta_description.strip()
    text = article.text.strip()

    # Set the subtitle to the description if it is empty
    if not subtitle:
        subtitle = description.strip()

    # Concatenate the extracted strings
    article_text = f"{title}\n\n{subtitle}\n\n{text}"

    # Tokenize the text into sentences
    sentences = sent_tokenize(article_text)

    stemmer = PorterStemmer()

    for sentence in sentences:
        # Tokenize the sentence into words
        words = word_tokenize(sentence)
        num_words = len(words)
        total_words += num_words
        
        # Identify the stop words in the sentence
        stop_words = set(stopwords.words('english'))
        filtered_words = [w for w in words if not w.lower() in stop_words]
        
        # Add the number of stop words and filtered sentence to the output
        num_stop_words = num_words - len(filtered_words)
        num_stop_words_per_sentence.append(num_stop_words)
        stop_words_per_sentence.append(filtered_words)
        filtered_sentences.append(" ".join(filtered_words))
        num_words_per_sentence.append(num_words)
        
        # Calculate the average number of stop words per sentence
        avg_stop_words_per_sentence.append(num_stop_words / num_words)

        # Perform stemming on each word using the Porter stemmer
        stemmed_words = [stemmer.stem(word) for word in words]

        # Combine the stemmed words back into a single string
        output_text = ' '.join(stemmed_words)

        doc = nlp(sentence)
        sentiment = doc.sentences[0].sentiment
        score = sentiment.score if hasattr(sentiment, 'score') else 0.0
        
        # Map the sentiment score to a label
        if score < 0.0:
            label = 'negative'
        elif score > 0.0:
            label = 'positive'
        else:
            label = 'neutral'
        
        # Add the sentence sentiment to the results list
        results.append((sentence, label, score))

        scores = analyzer.polarity_scores(sentence)
        score_list = [scores['neg'], scores['neu'], scores['pos']]
        scores_list.append(score_list)
    if not scores_list:
        return None
    
    for word in article_text.split():
        word = word.strip().lower()
        if word in lexicon.word.tolist():
            polarity = lexicon[lexicon.word == word].polarity.values[0]
            scores.append(polarity)
    
    # Calculate summary statistics
    num_stop_words = sum(num_stop_words_per_sentence)
    num_sentences = len(sentences)
    avg_stop_words_per_sentence_all = num_stop_words / num_sentences
    max_stop_words_per_sentence = max(num_stop_words_per_sentence)
    min_stop_words_per_sentence = min(num_stop_words_per_sentence)
    avg_stop_words_per_word = num_stop_words / sum(num_words_per_sentence)

    scores_array = np.array(scores_list)
    avg_scores = np.mean(scores_array, axis=0)
    max_scores = np.max(scores_array, axis=0)
    min_scores = np.min(scores_array, axis=0)
    std_scores = np.std(scores_array, axis=0)

    # Calculate statistics
    avg_score = np.mean(scores)
    max_score = np.max(scores)
    min_score = np.min(scores)
    sd_score = np.std(scores)
    
    # Calculate the average number of stop words per article
    avg_stop_words_per_sentence_avg = sum(avg_stop_words_per_sentence) / len(avg_stop_words_per_sentence)

    # initialize the sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    # analyze the sentiment of the text
    scores = analyzer.polarity_scores(article_text)

    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if not token in nltk.corpus.stopwords.words('english')]
    sentiwordnet_scores = []
    for token in tokens:
        pos_score = 0
        neg_score = 0
        synsets = swn.senti_synsets(token)
        for synset in synsets:
            pos_score += synset.pos_score()
            neg_score += synset.neg_score()
        if pos_score > neg_score:
          sentiment_score = 1
        elif neg_score > pos_score:
            sentiment_score = -1
        else:
            sentiment_score = 0
        sentiwordnet_scores.append(sentiment_score)
    assert(len(sentiwordnet_scores) == len(tokens))

    return {
        'article_text': article_text,
        'sentences': sentences,
        'num_stop_words': num_stop_words,
        "total_words": total_words,
        'avg_stop_words_per_sentence_all': avg_stop_words_per_sentence_all,
        'max_stop_words_per_sentence': max_stop_words_per_sentence,
        'min_stop_words_per_sentence': min_stop_words_per_sentence,
        'avg_stop_words_per_word': avg_stop_words_per_word,
        'avg_stop_words_per_sentence': avg_stop_words_per_sentence,
        'avg_stop_words_per_sentence_avg': avg_stop_words_per_sentence_avg,
        'filtered_sentences': filtered_sentences,
        'stop_words_per_sentence': stop_words_per_sentence,
        'num_words_per_sentence': num_words_per_sentence,
        'output_text': output_text
    }
    


In [11]:
results = preprocess_article(url)

print("Article")
print()
print(results["article_text"])
print()
print("Sentences")
print()
for sentence in results["filtered_sentences"]:
    print(sentence)
    print("Average number of stop words per sentence:", round(results["avg_stop_words_per_sentence"][results["filtered_sentences"].index(sentence)], 2))
    print()
for sentence in results["filtered_sentences"]:
        words = word_tokenize(sentence)
        print(words)

print("Statistics on stop words:")
print("Total number of words:", results["total_words"])
print("Number of stop words:", results["num_stop_words"])
print("Maximum number of stop words per sentence:", results["max_stop_words_per_sentence"])
print("Minimum number of stop words per sentence:", results["min_stop_words_per_sentence"])
print("Average number of stop words per article:", round(results["avg_stop_words_per_word"], 2))

print("Total number of words:", total_words)  # print the total sum of all words
print()
print("Stemmed text")
print()
print(results["output_text"])




UnboundLocalError: local variable 'total_words' referenced before assignment